In [1]:
import pymysql
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import scikit_posthocs as sp
import scipy.stats as stats
import csv

from statsmodels.stats.multicomp import MultiComparison

In [4]:
# tips = sns.load_dataset("tips")
# tips

In [2]:
con = pymysql.connect(host="localhost", user="gabic", passwd="1234", db="doenca_renal_cronica")
con.select_db('doenca_renal_cronica')
cursor = con.cursor()  
cursor.connection.autocommit(True)

In [3]:
#criando os dataframes por tabela 
dados_consulta = pd.read_sql('SELECT * FROM doenca_renal_cronica.dados_consulta',con=con)
cidade = pd.read_sql('SELECT * FROM doenca_renal_cronica.cidade',con=con)
ubs = pd.read_sql('SELECT * FROM doenca_renal_cronica.ubs',con=con)
aval_funcao_renal = pd.read_sql('SELECT * FROM doenca_renal_cronica.aval_funcao_renal',con=con)
consulta_has = pd.read_sql('SELECT * FROM doenca_renal_cronica.consulta_has',con=con)
consulta_drc = pd.read_sql('SELECT * FROM doenca_renal_cronica.consulta_drc',con=con)
consulta_dm = pd.read_sql('SELECT * FROM doenca_renal_cronica.consulta_dm',con=con)
calculo_tfg = pd.read_sql('SELECT * FROM doenca_renal_cronica.calculo_tfg',con=con)
preparacao_trs = pd.read_sql('SELECT * FROM doenca_renal_cronica.preparacao_trs',con=con)
raca = pd.read_sql('SELECT * FROM doenca_renal_cronica.raca',con=con)
instrucao = pd.read_sql('SELECT * FROM doenca_renal_cronica.instrucao',con=con)
paciente = pd.read_sql('SELECT * FROM doenca_renal_cronica.paciente',con=con)
exame_fisico = pd.read_sql('SELECT * FROM doenca_renal_cronica.exame_fisico',con=con)
avaliacao_tratamento = pd.read_sql('SELECT * FROM doenca_renal_cronica.avaliacao_tratamento',con=con)
medicamento = pd.read_sql('SELECT * FROM doenca_renal_cronica.medicamento',con=con)
complicacao_funcao_renal = pd.read_sql('SELECT * FROM doenca_renal_cronica.complicacao_funcao_renal',con=con)

#dicionario com todas tabelas
dic = {'dados_consulta':dados_consulta,'cidade':cidade,'ubs':ubs,'aval_funcao_renal':aval_funcao_renal,
       'consulta_has':consulta_has,'consulta_drc':consulta_drc,'consulta_dm':consulta_dm,'calculo_tfg':calculo_tfg,
       'preparacao_trs':preparacao_trs,'raca':raca,'instrucao':instrucao,'paciente':paciente,'exame_fisico':exame_fisico,
       'avaliacao_tratamento':avaliacao_tratamento,'medicamento':medicamento,'complicacao_funcao_renal':complicacao_funcao_renal
      }

In [4]:
df_num_dados_consulta = dados_consulta.select_dtypes(include = ['float64', 'int64'])
df_num_cidade = cidade.select_dtypes(include = ['float64', 'int64'])
df_num_ubs = ubs.select_dtypes(include = ['float64', 'int64'])
df_num_aval_funcao_renal = aval_funcao_renal.select_dtypes(include = ['float64', 'int64'])
df_num_consulta_has = consulta_has.select_dtypes(include = ['float64', 'int64'])
df_num_consulta_drc = consulta_drc.select_dtypes(include = ['float64', 'int64'])
df_num_consulta_dm = consulta_dm.select_dtypes(include = ['float64', 'int64'])
df_num_calculo_tfg = calculo_tfg.select_dtypes(include = ['float64', 'int64'])
df_num_preparacao_trs = preparacao_trs.select_dtypes(include = ['float64', 'int64'])
df_num_raca = raca.select_dtypes(include = ['float64', 'int64'])
df_num_instrucao = instrucao.select_dtypes(include = ['float64', 'int64'])
df_num_paciente = paciente.select_dtypes(include = ['float64', 'int64'])
df_num_exame_fisico = exame_fisico.select_dtypes(include = ['float64', 'int64'])
df_num_avaliacao_tratamento = avaliacao_tratamento.select_dtypes(include = ['float64', 'int64'])
df_num_medicamento = medicamento.select_dtypes(include = ['float64', 'int64'])
df_num_complicacao_funcao_renal = complicacao_funcao_renal.select_dtypes(include = ['float64', 'int64'])

dic_num = {'dados_consulta':df_num_dados_consulta,'cidade':df_num_cidade,'ubs':df_num_ubs,'afr':df_num_aval_funcao_renal,
       'has':df_num_consulta_has,'drc':df_num_consulta_drc,'dm':df_num_consulta_dm,'calculo_tfg':df_num_calculo_tfg,
       'preparacao_trs':df_num_preparacao_trs,'raca':df_num_raca,'instrucao':df_num_instrucao,'paciente':df_num_paciente,'exame_fisico':df_num_exame_fisico,
       'aval_tratamento':df_num_avaliacao_tratamento,'medicamento':df_num_medicamento,'cfr':df_num_complicacao_funcao_renal
      }

for tabela in dic_num:
    for coluna in dic_num[tabela].keys():
        i_split = coluna.split('_')
        if i_split[0]=='id':
            dic_num[tabela]=dic_num[tabela].drop(columns=[coluna], errors='ignore')
        if i_split[0]=='dt':
            dic_num[tabela]=dic_num[tabela].drop(columns=[coluna], errors='ignore')
        if i_split[0]=='total':
            dic_num[tabela]=dic_num[tabela].drop(columns=[coluna], errors='ignore')

In [5]:
#separando em uma matriz as todas colunas sem 0
# triglicerides_inicial_snulo = dic_num['cfr']['triglicerides_inicial']
# triglicerides_inicial_snulo = triglicerides_inicial_snulo [triglicerides_inicial_snulo.values != 0.0]
dic_snulo ={}
for tabela in dic_num:
    for coluna in dic_num[tabela]:
        copia = dic_num[tabela][coluna]
        copia= copia[copia.values != 0.0]
        dic_snulo[coluna] = copia

In [6]:
dic_snulo.keys()

dict_keys(['tempoAcomp', 'estagioI_alb', 'estagioF_alb', 'creatinina_inicial', 'creatinina_final', 'albumina_inicial', 'albumina_final', 'rel_alb_creat_uauc_inicial', 'rel_alb_creat_uauc_final', 'proteinuria24h_inicial', 'proteinuria24h_final', 'microAlbumi_inicial', 'microAlbumi_final', 'hematuria_inicial', 'hematuria_final', 'tshi_inicial', 'tshi_final', 'potassio_inicial', 'potassio_final', 'ureia_inicial', 'ureia_final', 'ureia24h_inicial', 'ureia24h_final', 'fosforo_inicial', 'fosforo_final', 'pth_intacto_inicial', 'pth_intacto_final', 'vitaminaD_inicial', 'vitaminaD_final', 'fosfatase_alc_inicial', 'fosfatase_alc_final', 'ck_inicial', 'ck_final', 'ecoae_inicial', 'ecoae_final', 'ecoao_inicial', 'ecoao_final', 'ecosiv_inicial', 'ecosiv_final', 'creatinina_1_2011', 'creatinina_2_2011', 'creatinina_1_2012', 'creatinina_2_2012', 'creatinina_1_2013', 'creatinina_2_2013', 'creatinina_1_2014', 'creatinina_2_2014', 'tfg_1_2011', 'tfg_2_2011', 'tfg_1_2012', 'tfg_2_2012', 'tfg_1_2013', 'tf

In [7]:
#estaísticas
for i in dic_snulo.keys():
    print('Tabela que está sendo analisada:'+ i)
    print(dic_snulo[i].describe())
    

Tabela que está sendo analisada:tempoAcomp
count    5054.000000
mean       17.764939
std        14.320183
min         1.000000
25%         5.000000
50%        14.000000
75%        29.000000
max        51.000000
Name: tempoAcomp, dtype: float64
Tabela que está sendo analisada:estagioI_alb
count    2482.000000
mean        1.433924
std         0.516423
min         1.000000
25%         1.000000
50%         1.000000
75%         2.000000
max         3.000000
Name: estagioI_alb, dtype: float64
Tabela que está sendo analisada:estagioF_alb
count    2133.000000
mean        1.383497
std         0.487315
min         1.000000
25%         1.000000
50%         1.000000
75%         2.000000
max         3.000000
Name: estagioF_alb, dtype: float64
Tabela que está sendo analisada:creatinina_inicial
count    6115.000000
mean        1.328741
std         0.817987
min         0.100000
25%         0.900000
50%         1.120000
75%         1.500000
max        20.500000
Name: creatinina_inicial, dtype: float64


count    1005.000000
mean       31.925224
std         4.717693
min         2.200000
25%        29.000000
50%        32.000000
75%        35.000000
max        52.000000
Name: ecoao_inicial, dtype: float64
Tabela que está sendo analisada:ecoao_final
count    165.000000
mean      32.424242
std        4.568118
min       18.000000
25%       29.000000
50%       32.000000
75%       35.000000
max       49.000000
Name: ecoao_final, dtype: float64
Tabela que está sendo analisada:ecosiv_inicial
count    996.000000
mean      10.248896
std        2.222749
min        0.900000
25%        9.000000
50%       10.000000
75%       12.000000
max       25.000000
Name: ecosiv_inicial, dtype: float64
Tabela que está sendo analisada:ecosiv_final
count    165.000000
mean      10.145455
std        2.223090
min        6.000000
25%        8.000000
50%       10.000000
75%       12.000000
max       18.000000
Name: ecosiv_final, dtype: float64
Tabela que está sendo analisada:creatinina_1_2011
count    1283.000000
mea

count    2727.0
mean        1.0
std         0.0
min         1.0
25%         1.0
50%         1.0
75%         1.0
max         1.0
Name: betabloq3, dtype: float64
Tabela que está sendo analisada:estatina9
count    3498.0
mean        1.0
std         0.0
min         1.0
25%         1.0
50%         1.0
75%         1.0
max         1.0
Name: estatina9, dtype: float64
Tabela que está sendo analisada:aas8
count    2830.0
mean        1.0
std         0.0
min         1.0
25%         1.0
50%         1.0
75%         1.0
max         1.0
Name: aas8, dtype: float64
Tabela que está sendo analisada:diur4
count    4422.0
mean        1.0
std         0.0
min         1.0
25%         1.0
50%         1.0
75%         1.0
max         1.0
Name: diur4, dtype: float64
Tabela que está sendo analisada:biguadina6
count    3295.0
mean        1.0
std         0.0
min         1.0
25%         1.0
50%         1.0
75%         1.0
max         1.0
Name: biguadina6, dtype: float64
Tabela que está sendo analisada:sulfonura7
count

In [8]:

with open('descricao_bd_sem_0.txt', 'w') as arquivo:
    for i in dic_snulo.keys():
        arquivo.write('Nome:'+ i +'\n')
        arquivo.write(str(dic_snulo[i].describe())+'\n')
        arquivo.write('\n')
    